In [3]:
import scipy as sp
import numpy as np
import pandas as pd
import timeit
import re
import json
import pickle
import fastparquet
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import datetime
import logging
import itertools

In [6]:
# rs = fastparquet.ParquetFile("DataStore/2018-10/Reg1016/naics_name_all_1016.parq").to_pandas()
# dm = "sim_docvecs"
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq")\
.to_pandas(["patent", "gyear"])
rs["op_gyear"] = rs["op"].map(dict(zip(pdf["patent"], pdf["gyear"])))

In [17]:
rs["gyear_match"] = (rs["tp_gyear"] == rs["op_gyear"])
cd = {"common_pat_inv": "Inventor Match", "primclass_match": "Primclass Match", "gyear_match": "Year Match"}

# Same NAICS
s1 = rs[[dm, "year_group"]].groupby("year_group").mean().rename(columns={"sim_docvecs": "NAICS Match"})
s2 = rs[[dm, "year_group"]].groupby("year_group").std().rename(columns={"sim_docvecs": "S.D."})
s1 = pd.concat([s1, s2], axis=1)
# Common inventor

for k in ["gyear_match", "primclass_match", "common_pat_inv",]:
    s2 = rs.loc[(rs[k] == True), [dm, "year_group"]].groupby("year_group").mean()\
    .rename(columns={"sim_docvecs":cd[k]})
    s3 = rs.loc[(rs[k] == True), [dm, "year_group"]].groupby("year_group").std()\
    .rename(columns={"sim_docvecs":"S.D."})
    s1 = pd.concat([s1, s2, s3], axis=1)
    
# s1 = s1.reset_index()
s1 = s1.rename(columns={"year_group": "Year Group"}).T
s1.index.name = None

In [18]:
s1

year_group,1975-85,1985-95,1995-05,2005-15
NAICS Match,0.124901,0.121715,0.127318,0.139251
S.D.,0.136074,0.133287,0.133765,0.135948
Year Match,0.124883,0.122010,0.127360,0.139984
S.D.,0.136582,0.134173,0.133641,0.136095
Primclass Match,0.189846,0.186469,0.195591,0.199946
S.D.,0.142823,0.143936,0.146470,0.146194
Inventor Match,0.310313,0.334446,0.322542,0.316685
S.D.,0.166998,0.164048,0.166608,0.169219
